In [1]:
import boto3
import pandas as pd

# Загрузка данных

In [2]:
bucket = 'LastFM-artist-recommender'
s3 = boto3.client(
   service_name='s3',
   endpoint_url='https://s3.storage.selcloud.ru'
)

In [3]:
for key in s3.list_objects(Bucket=bucket)['Contents']:
   print(key['Key'])

data
data/raw
data/raw/lastfm_artist_list.csv
data/raw/lastfm_user_scrobbles.csv


In [5]:
t = s3.get_object(Bucket=bucket, Key='data/raw/lastfm_artist_list.csv')['LastModified']

In [12]:
t.strftime("%m/%d/%Y_%H:%M:%S")

'05/18/2023_09:09:25'

In [9]:
datetime.datetime(2023, 5, 18, 9, 9, 25)

datetime.datetime(2023, 5, 18, 9, 9, 25)

In [8]:
import datetime
t == datetime.datetime(2023, 5, 18, 9, 9, 25)

False

In [42]:
artists = pd.read_csv(s3.get_object(Bucket=bucket, Key='data/raw/lastfm_artist_list.csv')['Body'])
scrobbles = pd.read_csv(s3.get_object(Bucket=bucket, Key='data/raw/lastfm_user_scrobbles.csv')['Body'])

# Анализ

## Артисты

In [43]:
artists.head(5)

,artist_id,artist_name
0,1,__Max__
1,2,_Algol_
2,3,-123 Min.
3,4,-Oz-
4,5,-T De Sangre


### Пропустки

In [44]:
artists['artist_id'].isna().sum()

0

In [45]:
artists['artist_name'].isna().sum()

0

### Дубликаты

In [46]:
print(f"Размер: {len(artists)}")
print(f"Уникальных артистов: {artists['artist_name'].nunique()}")

Размер: 17493
Уникальных артистов: 17492


In [47]:
artists[artists.duplicated(['artist_name'])]

,artist_id,artist_name
12095,12096,Red


In [48]:
artists[artists['artist_name'] == 'Red']

,artist_id,artist_name
12093,12094,Red
12095,12096,Red


## Прослушивания

In [49]:
scrobbles.head(5)

,user_id,artist_id,scrobbles
0,1,4562,13883
1,1,10191,11690
2,1,494,11351
3,1,6673,10300
4,1,8402,8983


### Проверим пользователей по количеству прослушанных артистов

In [50]:
scrobbles_by_users = scrobbles.groupby('user_id').count()
scrobbles_by_users.mean()

artist_id    49.044397
scrobbles    49.044397
dtype: float64

In [51]:
scrobbles_by_users[scrobbles_by_users['artist_id'] != 50]

,artist_id,scrobbles
user_id,,
8,49,49
33,15,15
36,49,49
108,1,1
151,49,49
...,...,...
1790,49,49
1830,49,49
1836,49,49


### Пропуски

In [52]:
scrobbles['user_id'].isna().sum()

0

In [53]:
scrobbles['artist_id'].isna().sum()

0

In [54]:
scrobbles['scrobbles'].isna().sum()

0

### Дубликаты

In [55]:
scrobbles[scrobbles.duplicated(['user_id', 'artist_id'])]

,user_id,artist_id,scrobbles
3202,65,857,63
8509,172,17063,28
8512,172,17110,17
9072,185,10311,964
10487,213,7314,126
...,...,...,...
83279,1700,472,138
84429,1724,11592,717
89391,1823,16174,138
89438,1824,8777,22
